# Import Required Libraries

In [1]:
import re
import pandas as pd
import numpy as np

# Import Dataset

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_dba9bbd52a9f4047affec71ab353fca2 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_dba9bbd52a9f4047affec71ab353fca2 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_dba9bbd52a9f4047affec71ab353fca2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9u2fwFmPJE5_GhH6YwbzXowP9iP2EVUI5S3ymY0SovLH',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_dba9bbd52a9f4047affec71ab353fca2)

body = client_dba9bbd52a9f4047affec71ab353fca2.get_object(Bucket='cellphonereview-donotdelete-pr-kmbgmt7fuv6gtt',Key='Reviews.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()


ModuleNotFoundError: No module named 'botocore'

# To Check FIrst 5 observations

In [ ]:
dataset.head()

# To Check Last 5 observations

In [ ]:
dataset.tail()

# Checking For Null Values

In [ ]:
dataset.isnull().any()

# Fill null values

In [ ]:
dataset['body'] = dataset['body'].fillna('').apply(str)
dataset['name'] = dataset['name'].fillna('').apply(str)
dataset['title'] = dataset['title'].fillna('').apply(str)
dataset['helpfulVotes'] = dataset['helpfulVotes'].fillna('').apply(str)

# Drop Null values

In [ ]:
dataset.isnull().sum()

# Dropping columns

In [ ]:
dataset=dataset.drop(columns=['asin','name','helpfulVotes','date'],axis=1)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.head()

In [ ]:
a=dataset['rating'].tolist()
a

### Based on the rating appending value 0 or 1 to the empty list

In [ ]:
d=[]
for i in range(len(a)):
    if a[i]>=3:
        d.append(1)
    else:
        d.append(0)
d

In [ ]:
dt=pd.DataFrame(d,columns=['emotion'])
dt

In [ ]:
data1=pd.concat([dataset,dt],axis=1)
data1.head()

### Drop verified column from data1

In [ ]:
data1.drop(['verified'],axis=1,inplace=True)

### Join both title and body

In [ ]:
data1['Review'] = data1[['title', 'body',]].agg(' '.join, axis=1)

In [ ]:
data1.head()

Dropping Columns

In [ ]:
data1.drop(['title','body','rating'],axis=1,inplace=True)

In [ ]:
data1.head()

In [ ]:
len(list(data1['emotion']))

split the data into x(independent variable)

In [ ]:
x=data1.iloc[:,1].values

In [ ]:
len(x)

In [ ]:
data1.head()

Check the data1 shape

In [ ]:
data1.shape

In [ ]:
#import natural language toolkit
import nltk
#import stopwords library to remove stopwords
from nltk.corpus import stopwords
#library used for stem the words
from nltk.stem.porter import PorterStemmer
#create an object for stemming
ps = PorterStemmer()
#library used for stem the words
from nltk.stem import WordNetLemmatizer
#create an object for wordnet lemmatizer
wordnet=WordNetLemmatizer()

In [ ]:
import re

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# Initialize empty array to append clean text  
corpus=[]
# no of rows to clean 
for i in range(len(x)):
    #replacing punctuations and numbers using re library 
    temp=re.sub('[^a-zA-Z]',' ',x[i])
    
    # convert all text to lower cases
    temp=temp.lower()
    # split to array(default delimiter is " ") 
    temp=temp.split()
    
    # creating WordNetLemmatizer object to take main lemma of each word
    wordnet = WordNetLemmatizer()
    #loop for leammatization each word in string array at ith row 
    temp=[wordnet.lemmatize(word) for word in temp if not word in set(
        stopwords.words('english'))]
    
    #rejoin all string array elements to create back into a string 
    temp=' '.join(temp)
    #append each string to create array of clean text 
    corpus.append(temp)

Before Stopwords: 
    "This is a sample sentence, showing off the stop words filtration."
    
After removing stopwords: 
    "This sample sentence, showing stop words filtration."

In [ ]:
corpus

In [ ]:
#creating bag of word model
from sklearn.feature_extraction.text import CountVectorizer
#To extract max 2000 feature, "max_features" is attribute to 
#experiment with to get better results
cv=CountVectorizer(max_features= 2000)
#z contains vectorized data (independent variable) 
z=cv.fit_transform(corpus).toarray()

Save the Bag of word model

In [ ]:
import pickle
pickle.dump(cv,open('count_vec.pkl','wb'))

In [ ]:
type(z)

In [ ]:
z

Creating dependent variable

In [ ]:
# y contains answers if review is positive or negative 
y=data1.iloc[:,0].values
y

In [ ]:
z.shape


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(z,y,test_size=0.3)

In [ ]:
x_train.shape

In [ ]:
#import library which uses tensorflow as backend
#import keras
#sequential is to intialise the model
from tensorflow.keras.models import Sequential
#Dense is used to add layers
from tensorflow.keras.layers import Dense

In [ ]:
model=Sequential()

Adding Output Layer

In [ ]:
model.add(Dense(2000,activation="relu"))
model.add(Dense(1000,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

compile the model

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

Training the Model

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=10)

Save the Model

In [ ]:
#this will save your model weights
#and h5 is the extension for keras
model.save('phone_ibm.h5')

In [ ]:
!tar -zcvf amazon_review.zip phone_ibm.h5

In [ ]:
ls -1

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"COTbPpFdckTWgzmx2njvA738MDHZ8voy_bMMnCSjt8ko"
}
client = APIClient(wml_credentials)

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
amazon_review_deploy

In [ ]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [ ]:
space_uid=guid_from_space_name(client,'amazon_review_deploy')
print("Space UID="+space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='amazon_review.zip',meta_props={
    client.repository.ModelMetaNames.NAME:"Amazonreviewmodel",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

In [ ]:
model_id